In [2]:
!pip install datasets evaluate accelerate
!pip install sacrebleu jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 47.4 MB/s eta 0:00:00


In [3]:
from huggingface_hub import interpreter_login
interpreter_login()

In [4]:
# Using it when you need to make it run automatically
# from huggingface_hub import login

# login(token="<Your hugging face write token>")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
import os
import random
import json
import torch
import torch.nn as nn
from functools import partial
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from huggingface_hub import HfApi

import evaluate
import numpy as np
from datasets import load_dataset
from transformers import Trainer
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer, TrainingArguments

import warnings
warnings.filterwarnings("ignore")

2024-05-27 18:50:27.891415: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 18:50:27.891509: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 18:50:28.023310: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Setup config

In [6]:
MODEL_NAME = "facebook/bart-large"
MAX_LENGTH = 512

# Download dataset & Preprocess


In [7]:
# Tải bộ dataset
dataset = load_dataset("grammarly/coedit")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 69071
    })
    validation: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 1712
    })
})

In [9]:
from datasets import Dataset

In [10]:
import pandas as pd
df1 = dataset['train'].to_pandas()
df2 = dataset['validation'].to_pandas()

df = pd.concat([df1,df2], axis = 0)

In [11]:
# df = dataset['train'].to_pandas()
df = df[df['task'] == 'gec'][['src',  'tgt']].reset_index().drop(columns = 'index')
df['src'] = df['src'].apply(lambda x: x.split(':')[1])
df = df.rename(columns = {'src':'input', 'tgt':'output'})
# dataset['train'] = Dataset.from_pandas(df.copy())

In [12]:
df

,input,output
0,"For example, countries with a lot of deserts ...","For example, countries with a lot of deserts c..."
1,"As the number of people grows, the need of ha...","As the number of people grows, the need for a ..."
2,Besides some technologically determinists tha...,Besides some technological determinists that a...
3,Safety is one of the crucial problems that ma...,Safety is one of the crucial problems that man...
4,On one hand more and more virus and hack can ...,"On the one hand, more and more viruses and hac..."
...,...,...
20303,"Jane and monica, I receive the letter, first ...","Dear Jane and Monica, I have received the lett..."
20304,"against THE CINEMA, WE HAVE SEEN A COMEDY ENT...","AT THE CINEMA, WE SAW A COMEDY ENTITLED "" AUTU..."
20305,The romance is just a small parts where Batma...,The romance is just a small part where Batman ...
20306,"Then, concerning a slight rising in 1992, it ...","Then, after a slight rise in 1992, they fell b..."


In [13]:
import pandas as pd
from tqdm import tqdm
import re

# Giả sử df là DataFrame đã được định nghĩa
# df = pd.read_csv("your_file.csv") hoặc bất kỳ phương thức nào bạn dùng để tạo DataFrame

def clean_text(content):
    if not isinstance(content, str):
        return None

    content = content.replace('..', '.') \
                     .replace('"', '') \
                     .replace(': .', '.') \
                     .replace('. .', '.') \
                     .replace("'", '') \
                     .replace('“', '') \
                     .replace('”', '')

#     # Check length of words
#     words = content.split()
#     for word in words:
#         if len(word) > 10:
#             return None

    if content and content[0] == '-':
        content = content.replace('-', '', 1)

    if content and content[0] == ' ':
        content = content[1:]

    content = content.replace('. - ', '. ') \
                     .replace('.. ', '... ') \
                     .replace('?.', '?') \
                     .replace(' ?.', '?') \
                     .replace('( ', '(') \
                     .replace(' )', ')') \
                     .replace('()', '') \
                     .replace('… ', '... ') \
                     .replace('….', '...') \
                     .replace('  ', ' ')

    if content.endswith(' .'):
        content = content[:-2] + '.'

    # Thay thế các ký tự không mong muốn bằng ký tự trống hoặc ký tự mong muốn
    content = content.replace('&', 'and')
    content = content.replace('…', '...')
    content = content.replace(':.', ':')
    
    # Xóa các ký tự không phải là ASCII
    content = ''.join([char for char in content if char.isascii()])

    # Xóa ký tự '·' ở đầu chuỗi nếu có
    if content.startswith('·'):
        content = content[1:]

    # Xóa khoảng trắng thừa
    content = ' '.join(content.split())

    match = re.search(r"\.\s\d+\.$", content)
    if match:
        content = re.sub(r"\.\s\d+\.$", ".", content)

    return content

# Use tqdm to create a progress bar
data = df['input']
data_processed = []

for content in tqdm(data):
    cleaned_content = clean_text(content)
    if cleaned_content:
        data_processed.append(cleaned_content)

# # Nếu muốn lưu lại kết quả vào DataFrame mới
# df_cleaned = pd.DataFrame(data_processed, columns=['input'])

# # Hoặc lưu vào cột mới của DataFrame hiện tại
# df['cleaned_input'] = data_processed


100%|██████████| 20308/20308 [00:00<00:00, 45117.98it/s]


In [14]:
len(data_processed)

20308

In [15]:
df['input'] = data_processed

In [16]:
# data_processed

In [17]:
df

,input,output
0,"For example, countries with a lot of deserts c...","For example, countries with a lot of deserts c..."
1,"As the number of people grows, the need of hab...","As the number of people grows, the need for a ..."
2,Besides some technologically determinists that...,Besides some technological determinists that a...
3,Safety is one of the crucial problems that man...,Safety is one of the crucial problems that man...
4,On one hand more and more virus and hack can a...,"On the one hand, more and more viruses and hac..."
...,...,...
20303,"Jane and monica, I receive the letter, first y...","Dear Jane and Monica, I have received the lett..."
20304,"against THE CINEMA, WE HAVE SEEN A COMEDY ENTI...","AT THE CINEMA, WE SAW A COMEDY ENTITLED "" AUTU..."
20305,The romance is just a small parts where Batman...,The romance is just a small part where Batman ...
20306,"Then, concerning a slight rising in 1992, it f...","Then, after a slight rise in 1992, they fell b..."


In [18]:
dataset['train'] = Dataset.from_pandas(df.copy())

In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 20308
    })
    validation: Dataset({
        features: ['_id', 'task', 'src', 'tgt'],
        num_rows: 1712
    })
})

In [20]:
dataset['test'] = dataset['train'].select(range(17000,20308))
dataset['validation'] = dataset['train'].select(range(14000,17000))
dataset['train'] = dataset['train'].select(range(14_000))

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 14000
    })
    validation: Dataset({
        features: ['input', 'output'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 3308
    })
})

In [22]:
dataset["train"][-9]

{'input': 'Seoul with Hiroaki Takeuchi, who is 26 years old, from 20th to 30th of next month.',
 'output': 'Seoul with Hiroaki Takeuchi, who is 26 years old, from the 20th to the 30th of next month.'}

# Tokenize dataset

In [23]:
# Example
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

input = dataset["train"][-1]['input']
output = dataset["train"][-1]['output']

# Sử dụng tokenizer để mã hóa dữ liệu đầu vào
inputs = tokenizer(
    input,
    text_target=output,
    max_length=MAX_LENGTH,
    truncation=True
)

inputs, len(inputs['input_ids'])

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

({'input_ids': [0, 10643, 768, 45, 7, 907, 24, 53, 7, 5956, 24, 15, 127, 308, 50, 19, 127, 964, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 10643, 768, 6, 45, 7, 907, 24, 53, 7, 5956, 24, 15, 127, 308, 50, 19, 127, 964, 4, 2]},
 20)

In [24]:
def preprocess_function(examples):
    # Tokenize the text and apply truncation
    return tokenizer(examples["input"],
                     text_target=examples["output"],
                     max_length=MAX_LENGTH,
                     truncation=True,
                     )

# Apply tokenization in a batched manner for efficiency
tokenized_datasets = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/14000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3308 [00:00<?, ? examples/s]

# Model

In [25]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

# Metric

In [26]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [27]:
import evaluate

metric = evaluate.load("sacrebleu")

In [28]:
predictions = [
    "I'm madly busy with work these days, but I think of you all the time."
]
references = [
    [
        "I'm madly busy with work these days, but I think of you all the time."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 100.00000000000004,
 'counts': [17, 16, 15, 14],
 'totals': [17, 16, 15, 14],
 'precisions': [100.0, 100.0, 100.0, 100.0],
 'bp': 1.0,
 'sys_len': 17,
 'ref_len': 17}

In [29]:
predictions = [
    "I'm madly busy for work these days but I think of you all the time."
]
references = [
    [
        "I'm madly busy with work these days, but I think of you all the time."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 64.52121005104219,
 'counts': [15, 12, 9, 6],
 'totals': [16, 15, 14, 13],
 'precisions': [93.75, 80.0, 64.28571428571429, 46.15384615384615],
 'bp': 0.9394130628134758,
 'sys_len': 16,
 'ref_len': 17}

In [30]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"sacrebleu": result["score"]}

# Fine-tuning the model

In [31]:
# Using it when you need to make it run automatically
# import wandb

# # Log in to wandb programmatically (if not already logged in via CLI)
# wandb.login(key="<Your wandb key>")

# # Initialize a new wandb run
# run = wandb.init(project="GEC-project")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hopmuc123456. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240527_185055-6pedljo0
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sandy-wildflower-30
wandb: ⭐️ View project at https://wandb.ai/hopmuc123456/GEC-project
wandb: 🚀 View run at https://wandb.ai/hopmuc123456/GEC-project/runs/6pedljo0


In [32]:

from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    do_train=True,
    do_eval=True,
    output_dir=f"model_v6", # where you storage the output model
    num_train_epochs=10,
#     learning_rate=1e-5,
    learning_rate=1e-6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="epoch",
    eval_steps=20_000,
    save_strategy="epoch",
    logging_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,  # Ensure the best model is loaded at the end
    metric_for_best_model="eval_loss",  # or another evaluation metric
    greater_is_better=False,  # Set to True if a higher metric value is better
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    weight_decay=0.001  # Apply weight decay
    
)

In [33]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, AutoModelForSeq2SeqLM, AutoTokenizer, TrainerCallback
import wandb

# Define a custom callback to log training loss
class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            wandb.log(logs)
            
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[LoggingCallback()]  # Add the custom logging callback
)

trainer.evaluate()

{'eval_loss': 1.9171562194824219,
 'eval_sacrebleu': 57.32505093324597,
 'eval_runtime': 246.5647,
 'eval_samples_per_second': 12.167,
 'eval_steps_per_second': 1.521}

In [34]:
# print("Tokenizer vocab size:", tokenizer.vocab_size)
# print("Problematic token ID:", 50249)

In [35]:
trainer.train()

Epoch,Training Loss,Validation Loss,Sacrebleu
1,0.594500,0.657931,63.254854
2,0.368100,0.596490,65.037280
3,0.342600,0.569003,65.476098
4,0.321500,0.570023,65.850989
5,0.308400,0.563707,66.226171
6,0.305100,0.565673,66.407674
7,0.290800,0.568180,66.470520
8,0.286800,0.566081,66.461962
9,0.279500,0.576106,66.510945
10,0.283200,0.564882,66.477142


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=8750, training_loss=0.3293835745675223, metrics={'train_runtime': 12622.5435, 'train_samples_per_second': 11.091, 'train_steps_per_second': 0.693, 'total_flos': 1.1923307937792e+16, 'train_loss': 0.3293835745675223, 'epoch': 10.0})

In [36]:
# trainer.evaluate()

In [37]:
trainer.push_to_hub(tags="text2text-generation", commit_message="version 2")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


events.out.tfevents.1716836118.ca283d3aebea.24.0:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sehilnlf/model_v6/commit/214ad7e537a5e4f51ae5589cff04d1abb9859953', commit_message='version 2', commit_description='', oid='214ad7e537a5e4f51ae5589cff04d1abb9859953', pr_url=None, pr_revision=None, pr_num=None)

# Inference

In [38]:
from transformers import pipeline

corrector = pipeline("text2text-generation", model="sehilnlf/model_v6")

config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [39]:
inputs = []
outputs = []
for i in range(10):
    inputs.append(dataset["test"][i*20]['input'])
    outputs.append([dataset["test"][i*20]['output']])

In [40]:
inputs.append('You am a boy.')

In [41]:
outputs.append(['You are a boy.'])

In [42]:
MAX_LENGTH = 512

# Define the text samples
texts = inputs.copy()

# Batch prediction
predictions = corrector(texts, max_length=MAX_LENGTH)

predicts = []
# Print predictions
for text, pred in zip(texts, predictions):
    predicts.append(pred['generated_text'])
    print("- " + pred['generated_text'])

- I'll appriciate your corrections, but probably I will not rewrite this until I will be able to write this correctly by myself.
- Many Japanese must work even at late night.
- The program merely broadcasts the scene of a running train with various music, but I can't resist gazing at it.
- I think with time, everything will be OK.
- I have just a cold.
- I have been looking forward to getting a junior girl colleague joining my department for 3 years.
- The first reason why we should make a judgment about a person's character by looking at his or her appearance or clothes is that we can get a lot of information about how he or she is from their appearance or clothing.
- I strongly believe, however, that companies should spend their budget on the protection of the environment for the following reasons:
- I heard this saying that Brazilian boys are born with a ball.
- Besides, the well had been dry for a long time, and it needed to be bridged.
- I am a boy.


In [43]:
predicts

["I'll appriciate your corrections, but probably I will not rewrite this until I will be able to write this correctly by myself.",
 'Many Japanese must work even at late night.',
 "The program merely broadcasts the scene of a running train with various music, but I can't resist gazing at it.",
 'I think with time, everything will be OK.',
 'I have just a cold.',
 'I have been looking forward to getting a junior girl colleague joining my department for 3 years.',
 "The first reason why we should make a judgment about a person's character by looking at his or her appearance or clothes is that we can get a lot of information about how he or she is from their appearance or clothing.",
 'I strongly believe, however, that companies should spend their budget on the protection of the environment for the following reasons:',
 'I heard this saying that Brazilian boys are born with a ball.',
 'Besides, the well had been dry for a long time, and it needed to be bridged.',
 'I am a boy.']

In [46]:
MAX_LENGTH = 512

# Define the text samples
texts = ['Hope you would be okay.']

# Batch prediction
predictions = corrector(texts, max_length=MAX_LENGTH)

predicts = []
# Print predictions
for text, pred in zip(texts, predictions):
    predicts.append(pred['generated_text'])
    print("- " + pred['generated_text'])

- I hope you will be okay.
